In [4]:
import pandas as pd

In [59]:
# Dates as the common index for all data
dates = [
    "7/31/2022",
    "8/31/2022",
    "9/30/2022",
    "10/31/2022",
    "11/30/2022",
    "12/31/2022",
    "1/31/2023",
    "2/28/2023",
    "3/31/2023",
    "4/30/2023",
    "5/31/2023",
    "6/30/2023",
    "7/31/2023",
]

# Data from the first image
light_passenger_vehicles_battery_electric = [
    186,
    1785,
    2353,
    980,
    1246,
    970,
    1955,
    1778,
    300,
    433,
    2558,
    2855,
    2419,
] * 12

# Data from the second image
light_goods_vehicles_battery_electric = [0, 0, 7, 0, 24, 32, 12, 34, 0, 0, 30, 32, 18] *12

# Data from the third image
light_goods_vehicles_all = [
    6491,
    6971,
    7607,
    6994,
    6887,
    6710,
    6216,
    4931,
    865,
    1662,
    7138,
    8757,
    6154,
] * 12

# Data from the fourth image
light_passenger_vehicles_all = [
    18315,
    21250,
    20107,
    18328,
    20998,
    17759,
    19373,
    14028,
    2827,
    6442,
    21862,
    25053,
    22227,
] * 12

# Combining the data into a single DataFrame
df = pd.DataFrame(
    {
        "Date": dates,
        "Light Passenger Vehicles Battery Electric": light_passenger_vehicles_battery_electric,
        "Light Goods Vehicles Battery Electric": light_goods_vehicles_battery_electric,
        "Light Goods Vehicles All": light_goods_vehicles_all,
        "Light Passenger Vehicles All": light_passenger_vehicles_all,
    }
)

df["Total Vehicles"] = (
    df["Light Passenger Vehicles All"] + df["Light Goods Vehicles All"]
)
df["Percent EV Passenger Vehicles"] = (
    df["Light Passenger Vehicles Battery Electric"]
    / df["Light Passenger Vehicles All"]
    * 100
)
df["Percent EV Goods Vehicles"] = (
    df["Light Goods Vehicles Battery Electric"] / df["Light Goods Vehicles All"] * 100
)
df["NSW Percent EV (Total)"] = (
    (
        df["Light Passenger Vehicles Battery Electric"]
        + df["Light Goods Vehicles Battery Electric"]
    )
    / df["Total Vehicles"]
    * 100
)

# create a new column with the date in the form of fraction of the year
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year + df["Date"].dt.dayofyear / 365
#df.to_csv("../data/processed/nsw_rego_data.csv", index=False)
df

ValueError: All arrays must be of the same length

In [ ]:
# transpose df with Date as the new column index
df_T = df.set_index("Date").T
display(df_T)

# revert to default index
df_T.reset_index(inplace=True)
display(df_T)

# split the index column into two columns named "Vehicle Type" and "Fuel Type"
df_T["Vehicle Type"] = [
    "Passenger" if "Passenger" in s else "Light Commercial" for s in df_T["index"]
]
df_T["Fuel Type"] = ["Electric" if "Electric" in s else "All" for s in df_T["index"]]

# drop rows which have "Total" or "Percent" in the "index" column
df_T = df_T[~df_T["index"].str.contains("Total")]
df_T = df_T[~df_T["index"].str.contains("Percent")]
df_T = df_T[~df_T["index"].str.contains("Year")]

# drop the "index" column
df_T.drop(columns=["index"], inplace=True)

# add a "State" column
df_T["State"] = ["NSW" for s in df_T["Vehicle Type"]]

# reorder the columns to have dates last
cols = df_T.columns.tolist()
cols = cols[-3:] + cols[:-3]
df_T = df_T[cols]

# loop over column names and if the name is in datetime format, change format to be YYYY-MM-DD
for col in df_T.columns:
    if isinstance(col, pd.Timestamp):
        df_T.rename(columns={col: col.strftime("%Y-%m-%d")}, inplace=True)


df_T.columns.name = "index"
df_T.to_csv("../data/processed/nsw_rego_data.csv", index=False)

display(df_T)

Date,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31
Light Passenger Vehicles Battery Electric,186.000000,1785.000000,2353.000000,980.000000,1246.000000,970.000000,1955.000000,1778.000000,300.000000,433.000000,2558.000000,2855.000000,2419.000000
Light Goods Vehicles Battery Electric,0.000000,0.000000,7.000000,0.000000,24.000000,32.000000,12.000000,34.000000,0.000000,0.000000,30.000000,32.000000,18.000000
Light Goods Vehicles All,6491.000000,6971.000000,7607.000000,6994.000000,6887.000000,6710.000000,6216.000000,4931.000000,865.000000,1662.000000,7138.000000,8757.000000,6154.000000
Light Passenger Vehicles All,18315.000000,21250.000000,20107.000000,18328.000000,20998.000000,17759.000000,19373.000000,14028.000000,2827.000000,6442.000000,21862.000000,25053.000000,22227.000000
Total Vehicles,24806.000000,28221.000000,27714.000000,25322.000000,27885.000000,24469.000000,25589.000000,18959.000000,3692.000000,8104.000000,29000.000000,33810.000000,28381.000000
Percent EV Passenger Vehicles,1.015561,8.400000,11.702392,5.347010,5.933898,5.462019,10.091364,12.674651,10.611956,6.721515,11.700668,11.395841,10.883160
Percent EV Goods Vehicles,0.000000,0.000000,0.092021,0.000000,0.348483,0.476900,0.193050,0.689515,0.000000,0.000000,0.420286,0.365422,0.292493
NSW Percent EV (Total),0.749819,6.325077,8.515552,3.870152,4.554420,4.094977,7.686897,9.557466,8.125677,5.343040,8.924138,8.538894,8.586731
Year,2022.580822,2022.665753,2022.747945,2022.832877,2022.915068,2023.000000,2023.084932,2023.161644,2023.246575,2023.328767,2023.413699,2023.495890,2023.580822


Date,index,2022-07-31 00:00:00,2022-08-31 00:00:00,2022-09-30 00:00:00,2022-10-31 00:00:00,2022-11-30 00:00:00,2022-12-31 00:00:00,2023-01-31 00:00:00,2023-02-28 00:00:00,2023-03-31 00:00:00,2023-04-30 00:00:00,2023-05-31 00:00:00,2023-06-30 00:00:00,2023-07-31 00:00:00
0,Light Passenger Vehicles Battery Electric,186.000000,1785.000000,2353.000000,980.000000,1246.000000,970.000000,1955.000000,1778.000000,300.000000,433.000000,2558.000000,2855.000000,2419.000000
1,Light Goods Vehicles Battery Electric,0.000000,0.000000,7.000000,0.000000,24.000000,32.000000,12.000000,34.000000,0.000000,0.000000,30.000000,32.000000,18.000000
2,Light Goods Vehicles All,6491.000000,6971.000000,7607.000000,6994.000000,6887.000000,6710.000000,6216.000000,4931.000000,865.000000,1662.000000,7138.000000,8757.000000,6154.000000
3,Light Passenger Vehicles All,18315.000000,21250.000000,20107.000000,18328.000000,20998.000000,17759.000000,19373.000000,14028.000000,2827.000000,6442.000000,21862.000000,25053.000000,22227.000000
4,Total Vehicles,24806.000000,28221.000000,27714.000000,25322.000000,27885.000000,24469.000000,25589.000000,18959.000000,3692.000000,8104.000000,29000.000000,33810.000000,28381.000000
5,Percent EV Passenger Vehicles,1.015561,8.400000,11.702392,5.347010,5.933898,5.462019,10.091364,12.674651,10.611956,6.721515,11.700668,11.395841,10.883160
6,Percent EV Goods Vehicles,0.000000,0.000000,0.092021,0.000000,0.348483,0.476900,0.193050,0.689515,0.000000,0.000000,0.420286,0.365422,0.292493
7,NSW Percent EV (Total),0.749819,6.325077,8.515552,3.870152,4.554420,4.094977,7.686897,9.557466,8.125677,5.343040,8.924138,8.538894,8.586731
8,Year,2022.580822,2022.665753,2022.747945,2022.832877,2022.915068,2023.000000,2023.084932,2023.161644,2023.246575,2023.328767,2023.413699,2023.495890,2023.580822


index,Vehicle Type,Fuel Type,State,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31
0,Passenger,Electric,NSW,186.0,1785.0,2353.0,980.0,1246.0,970.0,1955.0,1778.0,300.0,433.0,2558.0,2855.0,2419.0
1,Light Commercial,Electric,NSW,0.0,0.0,7.0,0.0,24.0,32.0,12.0,34.0,0.0,0.0,30.0,32.0,18.0
2,Light Commercial,All,NSW,6491.0,6971.0,7607.0,6994.0,6887.0,6710.0,6216.0,4931.0,865.0,1662.0,7138.0,8757.0,6154.0
3,Passenger,All,NSW,18315.0,21250.0,20107.0,18328.0,20998.0,17759.0,19373.0,14028.0,2827.0,6442.0,21862.0,25053.0,22227.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=75de7644-8bd4-4ecc-bdb2-2c9ef0ed94e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>